## Install

In [0]:
!pip install scrapy

In [0]:
!pip install pygsheets

In [0]:
!pip install unidecode

## Imports

In [0]:
import pandas as pd
import numpy as np
import pygsheets
from itertools import chain
import re
from pandas_datareader import data as web
import warnings
warnings.filterwarnings("ignore")


# Google Auth
import google.auth
from google.colab import auth
auth.authenticate_user()
credentials, _ = google.auth.default()
gs = pygsheets.client.Client(credentials)

# Import Custom Libraries
!cp "drive/My Drive/Colab Notebooks/financial_statements.py" .
!cp "drive/My Drive/Colab Notebooks/spreadsheet_func.py" .
from financial_statements import *
from spreadsheet_func import *


pd.options.display.float_format = '{0:,.2f}'.format

## Financial Statements

### Income Statement

In [0]:
inc_st = get_income_statement()

### Balance Sheets

In [0]:
b_sheet = get_balance_sheet(regroup=True)

### CashFlow Statement

In [0]:
# Regrouped Index
cf_st_reg = get_cashflow_statement(regroup=True)
# Original
cf_st = get_cashflow_statement(regroup=False)

### Sales by Segment

In [0]:
segments = get_segment_sales()

### Additional 10-K data & Aux Fields

In [0]:
# AAPL 2018 10-K info and AUX Fields
add_info = pd.DataFrame({
    'Depreciation and amortization pp&e': [-8300, -8200, -9300], # PP&E disclosure
    'DA related to PPE as pct of capex': 0.0,
    'DA not related to PPE': 0.0,
    'DA not related to PPE as pct of revenue': 0.0,
    'share_compensation_exp': cf_st.loc['Share-based compensation expense'].values,
    'Dividends':[-12188, -12803, -13735], # From 10-k disclosure
    'Repurchases': [-29000, -33001, -73056], # From 10-k disclosure
    'Minimum cash balance':0.0,
    'Cash from revolver':0.0, 
    'Discretionary Borrowing-Paydown':0.0,
    'Weighted average interest rate on commercial paper':[0.0, 0.012, 0.0218], # Commercial Paper disclosure
    'Interest expense Commercial Paper':0.0,
    'Weighted average interest rate on Long Term Debt':0.0,
    'Interest expense Long Term Debt':0.0, 
    'Weighted average interest rate on cash':[0.0173, 0.0199, 0.0216], # Other income/expense disclosure
    'Average share price':0.0, 
    'Share price increase':0.0, 
    'Impact of dilutive securities':0.0,
    'yoy pct growth in net income':0.0, 
    'AR as pct of sales':0.0, 
    'Days sales outstanding (DSO)':0.0, 
    'Inventory as pct of COGS':0.0, 
    'Inventory turnover':0.0, 
    'AP as pct of COGS':0.0, 
    'Days payables outstanding (DPO)':0.0,}, index = inc_st.columns.tolist()) 
    
add_info = add_info.T


In [0]:
# Growth rates and Other Calculations
other_cal = pd.DataFrame({
    'Revenue growth': 0.0,
    'Gross profit margin': 0.0,
    'RD pct of sales': 0.0,
    'SGA pct of sales': 0.0,
    'Tax rate': 0.0,
    'EBITDA': 0.0,
    'Adjusted EBITDA': 0.0,
    'Asset turnover': 0.0,
    'Net profit margin': 0.0,
    'ROA': 0.0,
    'ROE': 0.0}, index = inc_st.columns.to_list())

other_cal = other_cal.T


In [0]:
# AUX Fields for Products Forecasting 
products_cal = pd.DataFrame({
    'iPhone units growth':0.0,
    'iPad units growth':0.0,
    'Mac units growth':0.0,
    'ASP iPhone growth':0.0,
    'ASP iPad growth':0.0,
    'ASP Mac growth':0.0,
    'Services growth':0.0,
    'Other Products growth':0.0,
    'ASP iPhone':0.0,
    'ASP Mac':0.0,   
    'ASP iPad':0.0}, index = inc_st.columns.to_list())

products_cal = products_cal.T

## Connecting to Google Sheets

In [0]:
# Creating a Spreadsheet
sh = gs.open_by_url('https://docs.google.com/spreadsheets/d/1oLiIFFNvMJMZeQ2VnxjL87y4xQOg0CwDKgdUCr-ERFo/edit#gid=0')

# Connect to Worksheets
ws = sh.worksheet_by_title('FSM')


In [0]:
# # Delete All Named Ranges
# for i in ws.get_named_ranges():
#   ws.delete_named_range(re.compile(r'^(?:\S+\s){1}(\S+)').findall(str(i))[0])

## Copying Data & Modelling Spreadsheet

In [0]:
# Periods to forecast
per_est = [['2019P'],['2020P'],['2021P'],['2022P'],['2023P']]

# Dataframes to Copy in Worksheet 1
list_df = [inc_st, b_sheet, cf_st_reg, segments, add_info, other_cal, products_cal]


In [0]:
# Align Data in Spreadsheet
# align()

# Paste data into spreadsheet
paste_df(list_df)

# # Paste periods to estimate
estimated_periods(per_est,list_df)

# # Create named Ranges
create_ranges(list_df,per_est)


## Formulas & Data into the Spreadsheet

In [0]:
# Function to get all created ranges
get_ranges()

In [0]:
# Other income/(expense), net: Desegregated data from 10-K
rows = row_historical('interest_income','cells')
ws.update_values(crange=rows[0].label, values= [[3999, 5201, 	5686]])

rows = row_historical('interest_expense','cells')
ws.update_values(crange=rows[0].label, values= [[1456, 2323,	3240]])

rows = row_historical('other_expense_net','cells')
ws.update_values(crange=rows[0].label, values= [[1195, 133,	441]])


In [0]:
# Income Statement Calculations
cell_operation('gross_margin',['net_sales','gross_profit_margin'],sign='*')

cell_operation('cost_of_sales',['net_sales','gross_margin'],sign='-')

cell_operation('research_and_development', ['rd_pct_of_sales','net_sales'], sign='*')

cell_operation('selling_general_and_administrative', ['sga_pct_of_sales','net_sales'], sign='*') 

cell_operation('total_operating_expenses', 
              ['research_and_development','selling_general_and_administrative'],sign=' + ')

cell_operation('operating_income', ['gross_margin','-total_operating_expenses'],sign=' + ')

cell_operation('income_before_provision_for_income_taxes', 
              ['operating_income','interest_income','- other_expense_net','- interest_expense'], sign=' + ')

cell_operation('other_incomeexpense_net', ['- interest_expense','interest_income','- other_expense_net'], ' + ')

cell_operation('provision_for_income_taxes', 
              ['tax_rate','income_before_provision_for_income_taxes'], sign=' * ')

cell_operation('net_income', ['income_before_provision_for_income_taxes','-provision_for_income_taxes'], sign=' + ')


In [0]:
# Balance Sheet Calculations
cell_operation('total_assets', 
               ['cash__equivalents_st_and_lt_marketable_securities','accounts_receivable_net',\
                  'inventories','other_current_assets_inc_vendor_nt_receivables',\
                  'Property_plant_and_equipment_net','other_non_current_assets'],
               sign=' + ')

cell_operation('total_liabilities', 
               ['accounts_payable','other_current_liabilities','deferred_revenue_current_and_non_current',\
                  'commercial_paper','long_term_debt_inc_current_and_non_current','other_non_current_liabilities'],
              sign=' + ')

cell_operation('total_shareholders_equity', 
               ['common_stock','retained_earnings','accumulated_other_comprehensive_incomeloss'],
               sign=' + ')

cell_operation('total_liabilities_and_shareholders_equity', 
               ['total_liabilities','total_shareholders_equity'],
               sign=' + ')

inplace_cell_operation('common_stock', 'share_based_compensation_expense', sign='+' )


In [0]:
# Cashflow Statement
items = ['net_income','depreciation_and_amortization','share_based_compensation_expense',
         'decreases__increases_in_working_capital_assets','increases__decreases_in_working_capital_liabilities',
         'other_non_current_assets_cf','other_non_current_liabilities_cf']

cell_operation('cash_generated_by_operating_activities',items,'+')

cell_operation('cash_generated_byused_in_investing_activities',
                'payments_for_acquisition_of_property_plant_and_equipment','')

items = ['payments_for_dividends_and_dividend_equivalents','repurchases_of_common_stock',
         'change_in_commercial_paper_net','long_term_debt']

cell_operation('cash_used_in_financing_activities',items,'+')

items = ['cash_generated_by_operating_activities',
         'cash_generated_byused_in_investing_activities',
         'cash_used_in_financing_activities']

cell_operation('increasedecrease_in_cash_and_cash_equivalents',items,'+')


### Updating Historical Columns

In [0]:
# EBITDA
cell_historical('ebitda',cf_st.loc['Depreciation and amortization'].values,'operating_income',' + ')

# Adjusted EBITDA
cell_historical('adjusted_ebitda',cf_st.loc['Share-based compensation expense'].values,'ebitda',' + ')


In [0]:
# DA related to PPE as pct of capex
cell_historical('da_related_to_ppe_as_pct_of_capex','depreciation_and_amortization_ppe',
     cf_st.loc['Payments for acquisition of property, plant and equipment'].values, sign=' / ')

# DA not related to PPE
cell_historical('da_not_related_to_ppe', cf_st.loc['Depreciation and amortization'].values,
                                                    'depreciation_and_amortization_ppe',' + ')

# DA not related to PPE as % Net Sales
cell_historical('da_not_related_to_ppe_as_pct_of_revenue', 'da_not_related_to_ppe', 'net_sales', sign='/')


In [0]:
# Gross profit margin
cell_historical('gross_profit_margin','gross_margin','net_sales','/')

# RD pct of sales
cell_historical('rd_pct_of_sales','research_and_development', 'net_sales', '/')

# SGA pct of sales
cell_historical('sga_pct_of_sales','selling_general_and_administrative', 'net_sales', '/')

# Tax rate
cell_historical('tax_rate', 'provision_for_income_taxes', 'income_before_provision_for_income_taxes', '/')

# Revenue growth
pct_change('revenue_growth','net_sales','historical')

# Ratios
cell_historical('asset_turnover','total_assets','net_sales',' / ')
cell_historical('net_profit_margin','net_income','net_sales',' / ')
cell_historical('roa','net_income','total_assets',' / ')
cell_historical('roe','net_income','total_shareholders_equity',' / ')

# correct DIV Error
row_historical('roa','cells')[0].set_value('') 
row_historical('roe','cells')[0].set_value('') 


<Cell B113 ''>

## Forecasting

#### Equity Research

In [0]:
# Income Statement Projections
update_named_range('gross_profit_margin', [0.378, 0.381, 0.382, 0.382, 0.382])
update_named_range('rd_pct_of_sales', [0.062, 0.063, 0.063, 0.063, 0.063 ])
update_named_range('sga_pct_of_sales', [0.074, 0.069, 0.069, 0.069, 0.069  ])
update_named_range('tax_rate', [0.167, 0.17, 0.169, 0.169, 0.169 ])


# CAPEX Projections
update_named_range('payments_for_acquisition_of_property_plant_and_equipment', [-13285, -13649, -13819, -14773, -15792 ])


#### Grow inline with Revenue Growth

In [0]:
# Grow inline with Revenue Growth
rg_list = ['net_sales','other_current_assets_inc_vendor_nt_receivables',
           'other_non_current_assets','other_current_liabilities','deferred_revenue_current_and_non_current',
           'other_non_current_liabilities','share_compensation_exp']
      
cell_growth(rg_list, variable='revenue_growth')

cell_operation('share_based_compensation_expense','share_compensation_exp','')


#### Grow inline with COGS

In [0]:
# Grow inline with COGS Growth
# cg_list = ['inventories','accounts_payable']

# inplace_growth(cg_list, variable='cost_of_sales') # Elimiar la formula 'inplace_growth'

#### Straight line

In [0]:
# Straight line
sl_list = ['other_expense_net','dividends','repurchases','long_term_debt_inc_current_and_non_current',
           'accumulated_other_comprehensive_incomeloss', 'weighted_average_interest_rate_on_commercial_paper',
           'weighted_average_interest_rate_on_cash']

straight_line(sl_list)


### Scheadules

#### Working Capital

In [0]:
# Accounts Receivable
# Update Historical
cell_historical('ar_as_pct_of_sales','accounts_receivable_net','net_sales',' / ')

var = row_historical('ar_as_pct_of_sales','cells')
target = row_historical('days_sales_outstanding_dso','cells')
tup = list(zip(target,var))

for i in range(len(tup)):
  tup[i][0].formula = tup[i][1].label + '* 365'

# Average the last two periods
cells = named_range('ar_as_pct_of_sales','cells')
for i in cells:
  i.formula = 'AVERAGE('+ cells[0].neighbour((0,-2)).label + ':' + cells[0].neighbour('left').label + ')'

# Days in Estimated Periods
cells = named_range('days_sales_outstanding_dso','cells')
for i in cells:
  i.formula = 'ar_as_pct_of_sales' + ' * 365'

# Computing AR
cell_operation('accounts_receivable_net',['net_sales','ar_as_pct_of_sales'],' * ')


In [0]:
# Inventory
# Update Historical
cell_historical('inventory_as_pct_of_cogs','inventories','cost_of_sales',' / ')
cell_historical('inventory_turnover','cost_of_sales','inventories',' / ')
row_historical('inventory_turnover','cells')[0].set_value('') # correct DIV Error

# Average the last two periods
cells = named_range('inventory_as_pct_of_cogs','cells')
for i in cells:
  i.formula = 'AVERAGE('+ cells[0].neighbour((0,-2)).label + ':' + cells[0].neighbour('left').label + ')'

# Inventory Turnover
cell_operation('inventory_turnover',['cost_of_sales','inventories'],'/')

# Compute Inventory
cell_operation('inventories',['cost_of_sales','inventory_as_pct_of_cogs'],' * ')


In [0]:
# Accounts Payble
# Update Historical
cell_historical('ap_as_pct_of_cogs','accounts_payable','cost_of_sales',' / ')

var = row_historical('ap_as_pct_of_cogs','cells')
target = row_historical('days_payables_outstanding_dpo','cells')
tup = list(zip(target,var))

for i in range(len(tup)):
  tup[i][0].formula = tup[i][1].label + '* 365'

# Average the last two periods
cells = named_range('ap_as_pct_of_cogs','cells')
for i in cells:
  i.formula = 'AVERAGE('+ cells[0].neighbour((0,-2)).label + ':' + cells[0].neighbour('left').label + ')'

# Days in Estimated Periods
cells = named_range('days_payables_outstanding_dpo','cells')
for i in cells:
  i.formula = 'ap_as_pct_of_cogs' + ' * 365'

# Computing AP
cell_operation('accounts_payable',['cost_of_sales','ap_as_pct_of_cogs'],' * ')


#### PP&E

In [0]:
# Last Row in Worksheet
start, pos = 1, [1]
for i in list_df:
    start = start + len(i) + 3
    pos.append(start)

# Create Variable
ws.update_value('A'+ str(pos[-1]), 'Depreciation Factor')
# Name of Variable
dep_factor = 'depreciation_factor'
ws.create_named_range(dep_factor, start='B'+ str(pos[-1]),end='B'+ str(pos[-1]),grange=0)
# Update Value
ws.get_named_range(dep_factor).update_values([[0.02]])


In [0]:
# DA related to PPE as pct of capex
inplace_cell_operation('da_related_to_ppe_as_pct_of_capex',dep_factor,' + ')

# Depreciation and amortization pp&e
cell_operation('depreciation_and_amortization_ppe',
               ['payments_for_acquisition_of_property_plant_and_equipment','da_related_to_ppe_as_pct_of_capex'],
               '*')

# DA not related to PPE
straight_line('da_not_related_to_ppe_as_pct_of_revenue')
cell_operation('da_not_related_to_ppe',['da_not_related_to_ppe_as_pct_of_revenue','net_sales'],'*')


In [0]:
# Cashflow: Depreciation and Amortization
cell_operation('depreciation_and_amortization',
               ['-depreciation_and_amortization_ppe','da_not_related_to_ppe'],'+')


# Balance Sheets: Property, plant and equipment, net
items = ['- payments_for_acquisition_of_property_plant_and_equipment','depreciation_and_amortization_ppe']
inplace_cell_operation('property_plant_and_equipment_net', items, ' + ')


#### Other non-current assets

In [0]:
cell_last_per_operation('other_non_current_assets_cf','other_non_current_assets',
                        ['other_non_current_assets','da_not_related_to_ppe'],
                        sign=' - ')

#### Retained Earnings

In [0]:
inplace_cell_operation('retained_earnings',['net_income','dividends','repurchases'],sign=' + ')

# To Cashflow
cell_operation('payments_for_dividends_and_dividend_equivalents','dividends','')
cell_operation('repurchases_of_common_stock','repurchases','')


#### The Revolving Credit Facility

In [0]:
# Minumun Cash Balance Assumptions
update_named_range('minimum_cash_balance', [-50000]*5)


In [0]:
# # Cash available (needed) to pay down (draw from) revolver
cell_last_per_operation('cash_from_revolver','cash__equivalents_st_and_lt_marketable_securities',
                        ['minimum_cash_balance','increasedecrease_in_cash_and_cash_equivalents',
                         '- change_in_commercial_paper_net'],' + ')


In [0]:
# Discretionary borrowing/paydown
update_named_range('discretionary_borrowing_paydown',[12000]*5)

# Revolver
revolver('commercial_paper','cash_from_revolver','discretionary_borrowing_paydown')


#### Interest Expense

##### Update Historicals

In [0]:
# Historical Interest Expense for Commercial Paper

historical_interest_expense('interest_expense_commercial_paper',
                            'commercial_paper','weighted_average_interest_rate_on_commercial_paper')


In [0]:
# Historical Interest Expense for Long Term Debt
cell_historical('interest_expense_long_term_debt','interest_expense','interest_expense_commercial_paper',' - ')


In [0]:
# Historical Weighted Average Rate for Long Term Debt
historical_weighted_avg_interest_rate('weighted_average_interest_rate_on_long_term_debt',
                                      'long_term_debt_inc_current_and_non_current',
                                      'interest_expense_long_term_debt')


##### Forecasting

In [0]:
# Forecasting Weighted Average Rate for Long Term Debt
straight_line('weighted_average_interest_rate_on_long_term_debt')

# Commercial Paper Interest Expense
avg_last_per('interest_expense_commercial_paper','commercial_paper','weighted_average_interest_rate_on_commercial_paper')

# Long Term Debt Interest Expense
avg_last_per('interest_expense_long_term_debt',
             'long_term_debt_inc_current_and_non_current','weighted_average_interest_rate_on_long_term_debt')

# Total Interest Expense
cell_operation('interest_expense',['interest_expense_long_term_debt','interest_expense_commercial_paper'],' + ')


#### Interest Income

In [0]:
avg_last_per('interest_income','cash__equivalents_st_and_lt_marketable_securities','weighted_average_interest_rate_on_cash')


### Statements Reconciliation

In [0]:
# EBITDA and Adjusted EBITDA
cell_operation('ebitda',['operating_income','depreciation_and_amortization'],' + ')
cell_operation('adjusted_ebitda',['ebitda','share_based_compensation_expense'],' + ')

# Balance: Cash and Equivalents
inplace_cell_operation('cash__equivalents_st_and_lt_marketable_securities',
                       'increasedecrease_in_cash_and_cash_equivalents',' + ')

# Change in Commercial Paper
diff_abs('change_in_commercial_paper_net','commercial_paper')

# Change in Long Term Debt
diff_abs('long_term_debt','long_term_debt_inc_current_and_non_current')


In [0]:
# Cashflow: Other non-current liabilities
diff_abs('other_non_current_liabilities_cf','other_non_current_liabilities')

# Cashflow: Decreases / (Increases) in working capital assets
diff_abs('decreases__increases_in_working_capital_assets',
         ['accounts_receivable_net','inventories','other_current_assets_inc_vendor_nt_receivables'])

# Multiply by -1: For cashflow is a negative cash flow
for i in named_range('decreases__increases_in_working_capital_assets','cells'):
  i.formula = '(' + i.formula.replace('=','') + ')* - 1'

# Cashflow: Increases / (Decreases) in working capital liabilities
diff_abs('increases__decreases_in_working_capital_liabilities',
         ['accounts_payable','other_current_liabilities','deferred_revenue_current_and_non_current'])


In [0]:
# Ratios
# Forecast Period: Computing Ratios
cell_operation('asset_turnover',['total_assets','net_sales'],' / ')
cell_operation('net_profit_margin',['net_income','net_sales'],' / ')
cell_operation('roa',['net_income','total_assets'],' / ')
cell_operation('roe',['net_income','total_shareholders_equity'],' / ')


In [0]:
# Balance Check
balance_check(list_df)

# Historical Columns
cell_historical('balance_check','total_assets','total_liabilities_and_shareholders_equity',' - ')


### Forecasting EPS

In [0]:
# Average Stock Price From Stock Data
df = web.DataReader('AAPL', 'yahoo',start='2018',end='2019')['Adj Close']
avg_stock_price = df.mean()
# Update Mean in Cell
stock = named_range('average_share_price','cells')[0].label
ws.cell(stock).set_value(avg_stock_price)


<Cell E90 184.07218519719947>

In [0]:
# Net Income Pct Change
pct_change('yoy_pct_growth_in_net_income','net_income','historical')
pct_change('yoy_pct_growth_in_net_income','net_income','forecast')

# Stock Price Forecasting
target = named_range('average_share_price','cells')[1:]
for i in target:
  i.formula = i.neighbour('left').label + str(' * (1 + ') + 'yoy_pct_growth_in_net_income' + ')'

# Share price Increase/Decrease = YOY % growth in Net Income
cell_operation('share_price_increase','yoy_pct_growth_in_net_income','')


In [0]:
# Computing Basic Shares
cells = named_range('basic_in_shares','cells')

for i in cells:
  i.formula = i.neighbour('left').label + ' + ('+'share_based_compensation_expense' +\
            ' - repurchases_of_common_stock'+')/'+'average_share_price'

# Computing Basic EPS
cell_operation('basic_in_dollars_per_share',['net_income','basic_in_shares'],' / ')

# Multiply by 1000
for i in named_range('basic_in_dollars_per_share','cells'):
  i.formula = '(' + i.formula.replace('=','') + ')* 1000'


In [0]:
# Computing Historical: Impact of dilutive securities  
cell_historical('impact_of_dilutive_securities','diluted_in_shares','basic_in_shares',' - ')

# Computing Diluted Shares
cell_last_per_operation('diluted_in_shares','impact_of_dilutive_securities','basic_in_shares',' + ')

# Computing Diluted EPS
cell_operation('diluted_in_dollars_per_share',['net_income','diluted_in_shares'],' / ')

# Multiply by 1000
for i in named_range('diluted_in_dollars_per_share','cells'):
  i.formula = '(' + i.formula.replace('=','') + ')* 1000'

# Forecast Calculation: Impact of dilutive securities
cell_operation('impact_of_dilutive_securities',['diluted_in_shares','basic_in_shares'],' - ')


### Forecasting Revenue

In [0]:
# Update Historicals
pct_change('iphone_units_growth','iphone_units','historical')
pct_change('ipad_units_growth','ipad_units','historical')
pct_change('mac_units_growth','mac_units','historical')

# Historical Average Selling Price (ASP)
cell_historical('asp_iphone','iphone_1','iphone_units',' / ')
cell_historical('asp_ipad','ipad_1','ipad_units',' / ')
cell_historical('asp_mac','mac_1','mac_units',' / ')

# ASP:  Multiply by 1000
rg = ['asp_iphone','asp_ipad','asp_mac']

for i in rg:
  for ii in row_historical(i,'cells'):
    ii.formula = ii.formula.replace('=','') + '* 1000'

# ASP Historical Growth
pct_change('asp_iphone_growth','asp_iphone','historical')
pct_change('asp_ipad_growth','asp_ipad','historical')
pct_change('asp_mac_growth','asp_mac','historical')

# Historical: Other Products and Services Growth
pct_change('services_growth','services_2','historical')
pct_change('other_products_growth','other_products_13','historical')


In [0]:
# Forecasting Growth Units
update_named_range('iphone_units_growth',[-0.20, 0.09, 0.01, 0, 0 ])
update_named_range('ipad_units_growth', [0.06, 0.067, 0.01, 0, 0 ])
update_named_range('mac_units_growth', [0.08, -0.03, 0, 0, 0 ])

# Forecasting ASP Growth
update_named_range('asp_iphone_growth', [0.035, -0.09, 0, 0, 0 ])
update_named_range('asp_ipad_growth', [0.02, -0.05, 0, 0, 0 ])
update_named_range('asp_mac_growth', [-0.04, 0.03, 0, 0, 0 ])

# Other Products and Services Growth
update_named_range('other_products_growth', [0.32, 0.15, 0.15, 0.15, 0.15])
update_named_range('services_growth', [0.23, 0.17, 0.17, 0.17, 0.17 ])


In [0]:
# Computing Forecast Periods
# ASP
cell_growth('asp_iphone','asp_iphone_growth')
cell_growth('asp_ipad','asp_ipad_growth')
cell_growth('asp_mac','asp_mac_growth')
cell_growth('other_products_13','other_products_growth')
cell_growth('services_2','services_growth')

# Units
cell_growth('iphone_units','iphone_units_growth')
cell_growth('ipad_units','ipad_units_growth')
cell_growth('mac_units','mac_units_growth')

# Product Sales
cell_operation('iphone_1',['iphone_units','asp_iphone'],' * ')
cell_operation('ipad_1',['ipad_units','asp_ipad'],' * ')
cell_operation('mac_1',['mac_units','asp_mac'],' * ')

# Product Sales: Divide Outcome by 1000
rg = ['iphone_1','ipad_1','mac_1']

for i in rg:
  for ii in named_range(i,'cells'):
    ii.formula = ii.formula.replace('=','') + '/ 1000'

# Total Net Sales
cell_operation('total_net_sales',
               ['iphone_1','ipad_1','mac_1','other_products_13','services_2'],' + ')

# Net Sales Growth (Revenue Growth)
pct_change('revenue_growth','total_net_sales','forecast')


## CAGR

In [0]:
cagr(['net_sales','gross_margin','operating_income',
      'income_before_provision_for_income_taxes','net_income'],per_est)
